## Importing all required libraries

In [ ]:
import numpy as np
import xarray as xr

import rasterio.features
import stackstac
import pystac_client
import planetary_computer

import xrspatial.multispectral as ms

from dask_gateway import GatewayCluster

## Create a Dask cluster

In [ ]:
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.

client = cluster.get_client()

cluster.adapt(minimum=4, maximum=24)
print(cluster.dashboard_link)

In [ ]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
[
            [
              28.184331178064923,
              -24.68487285649138
            ],
            [
              28.184331178064923,
              -24.803760340510735
            ],
            [
              28.346284282010146,
              -24.803760340510735
            ],
            [
              28.346284282010146,
              -24.68487285649138
            ],
            [
              28.184331178064923,
              -24.68487285649138
            ]
          ]
    ],
}
bbox = rasterio.features.bounds(area_of_interest)


In [ ]:
stac = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = stac.search(
    bbox=bbox,
    datetime="2021-12-31/2023-01-01",
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 25}},
)

items = search.item_collection()
# print(len(items))

print(items.items[0].assets)

In [ ]:
data = (
    stackstac.stack(
        items,
        assets=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09", "B8A", "B11", "B12"],
        chunksize=4096,
        resolution=100,
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
    .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
)
data

Since the data matching our query isn't too large we can persist it in distributed memory. Once in memory, subsequent operations will be much faster.

In [ ]:
data = data.persist()
data

## Create a median composite

In [ ]:
median = data.median(dim="time").compute()

In [ ]:
median

In [ ]:
image = ms.true_color(*median)  # expects red, green, blue DataArrays
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))

# ax.set_axis_off()
image.plot.imshow(ax=ax);

## Monthly composite

In [ ]:
monthly = data.groupby("time.month").median().compute()

images = [ms.true_color(*x) for x in monthly]
images = xr.concat(images, dim="time")



In [ ]:
g = images.plot.imshow(x="x", y="y", rgb="band", col="time", col_wrap=3, figsize=(6, 8))
for ax in g.axes.flat:
    ax.set_axis_off()

plt.tight_layout()